# Week 3 Assignment #

## Create Toronto data frame and Submit a link to your Notebook on your Github repository. (10 marks) ##

In [1]:
# import necessary packages
import pandas as pd

**First read the table from wiki**

In [2]:
# wikipedia url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**Extract the table**

**Read html table into pandas dataframe**

In [3]:
# Read html table into pandas dataframe
my_list = pd.read_html(url, header=0,match='Postcode')
df = my_list[0]

In [4]:
# drop the rows with borough not assigned
df = df[df['Borough']!='Not assigned']

In [5]:
# sort data drame by postal codes
df=df.sort_values(by=['Postcode'])

In [6]:
# initialize another dataframe with first row of the table sorted by postal code
df2 = pd.DataFrame(df.iloc[0]).transpose()

In [7]:
# loop through the sorted table
j = 1
for i in range(1, df.shape[0]):
    # if the postal code is same as the last postal code, append the neighbourhood
    if df.iloc[i]['Postcode'] == df2.iloc[j-1]['Postcode']:
        df2.iloc[j-1]['Neighbourhood'] = df2.iloc[j-1]['Neighbourhood'] + ', ' + df.iloc[i]['Neighbourhood']
    # otherwise add the row to the new dataframe
    else:
        df2 = df2.append(pd.DataFrame(df.iloc[i]).transpose())
        j = j + 1 

In [8]:
# if the neighbourhood is not assigned, assigned the borough value in the same row to neighbourhood
df2.loc[df2['Neighbourhood'] == 'Not assigned','Neighbourhood']=df2.loc[df2['Neighbourhood'] == 'Not assigned','Borough']

In [9]:
df2.shape

(103, 3)

## Add geocode info and submit a link to the new Notebook on your Github repository. (2 marks) ##

In [10]:
# Read the geocode data
geocode_data = pd.read_csv('http://cocl.us/Geospatial_data')
print('geocode data read')

geocode data read


In [11]:
# Join the data frames to add the latitude and longitude by postal code
df3 = pd.merge(df2, geocode_data, how='left', left_on=['Postcode'], right_on=['Postal Code'])
df3.drop(['Postal Code'], axis=1)
df3.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Explore and cluster the neighborhoods in Toronto. (3 marks) ##

**Install and import the libraries**

In [13]:
# Install the libraries
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes


# Import the librarries
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  31.18 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  14.42 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  19.71 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  18.26 MB/s
Libraries imported.


**Create the map of TO**

In [18]:
TO_boroughs_data = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
TO_boroughs_data.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [19]:
# create map of Toronto using latitude and longitude values
map_TO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TO_boroughs_data['Latitude'], TO_boroughs_data['Longitude'], TO_boroughs_data['Borough'], TO_boroughs_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_TO

**Define Foursquare Credentials and Version**

In [20]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret
